In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import seaborn as sns
from ExplainableMaximumLikelihoodCalculator import ExplainableMaximumLikelihoodCalculator
import pysam
from Bio import SeqIO, Seq, SeqRecord, pairwise2
from Bio.pairwise2 import format_alignment
from LoadDataUtils import getListOfReadsFromBamFile, getListOfReadsFromFastaFile
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import math
import shap
from scipy.stats import chisquare
from scipy.special import rel_entr
fileNameSapiens =  "data/simulatedData/human_chinese_AF346973_500samples.fas" #File that contains 500 reads from homo sapiens
fileNameNeanderthals =  "data/simulatedData/Neanderthal_Goyet_KX198085_500samples.fas" #File that contains 500 reads from neanderthal
fileNameDenisovans =  "data/simulatedData/denisova_kx663333_500samples.fas" #File that contains 500 reads from denisovan
neanderthals_500_generated = getListOfReadsFromFastaFile(fileNameNeanderthals)
sapiens_500_generated = getListOfReadsFromFastaFile(fileNameSapiens)
denisovan_500_samples = getListOfReadsFromFastaFile(fileNameDenisovans)
path_to_frequencies_table = "data/substitution_matrix.tsv"

2023-07-16 16:33:32.743344: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-16 16:33:33.010264: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-16 16:33:33.068052: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-16 16:33:33.749156: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
sapiens_reference_file_names = [
                    "data/reference_files/human_AF346981_French.fa",
                     "data/reference_files/human_AY195760_Korea.fa",
                      "data/reference_files/human_AY882416_Ethiopia.fa",
                      "data/reference_files/human_AY963586_Italian.fa",
                      "data/reference_files/human_AY195781_Caucasian.fa",
                      "data/reference_files/human_AY195757_Iraqi-Israeli.fa",
                      "data/reference_files/human_AY195749_NativeAmerican.fa"]
neanderthals_reference_file_names = [
                            "data/reference_files/neanderthal_mezmaiskaya1_FM865411.fa",
                           "data/reference_files/Neanderthal_Altai_KC879692.fa",
                           "data/reference_files/Neanderthal_Denisova11_full_mtDNA_KU131206.fa",
                           "data/reference_files/Neanderthal_Spy_94a_MG025538.fa",
                            "data/reference_files/Neanderthal_Vindija33.16_AM948965.fa",
                            "data/reference_files/Neanderthal_Vindija33.19_KJ533545.fa"]
denisovan_reference_file_names = [  
                        "data/reference_files/Denisova_MT576653.1.fa",
                        "data/reference_files/Denisova_MT576652.1.fa",
                        "data/reference_files/Denisova_4_FR695060.fa",
                        "data/reference_files/Denisova_8_KT780370.fa",
                        "data/reference_files/Denisova_manual_phalanx_NC_013993.fa",
                        "data/reference_files/Denisova_MT576651.1.fa"]

In [3]:
def generate_sample(sap, nean, den):
    list_before =  list(np.random.choice(sapiens_500_generated, sap)) + list(np.random.choice(neanderthals_500_generated, nean)) + list(np.random.choice(denisovan_500_samples, den))
    l = []
    for i in list_before:
        l.append(str(i))
    return l

In [4]:
from itertools import combinations

def find_all_subsets(group):
    all_subsets = []
    n = len(group)
    
    for r in range(n + 1):
        subsets_r = combinations(group, r)
        all_subsets.extend(list(subsets_r))
        
    return all_subsets

In [10]:
def calculate_shapley_values_for_datum(data_point, length_of_data, ml_calculator):
    indexes_without_datum = [i for i in range(length_of_data) if i != data_point]
    all_subsets = find_all_subsets(indexes_without_datum)
    sum_over_all_subsets = 0 
    for G in all_subsets:
        g_with = [i for i in G]
        g_with.append(data_point)
        with_datum = ml_calculator.calc_maximum_likelihood_on_subset(subset_of_indexes=g_with).values[0]
        without_datum = ml_calculator.calc_maximum_likelihood_on_subset(subset_of_indexes=G).values[0]
        with_minus_without = with_datum - without_datum
        multiplier = np.math.factorial(len(G)) * np.math.factorial(length_of_data - len(G) - 1)
        value_for_sum = with_minus_without * multiplier 
        sum_over_all_subsets += value_for_sum
    shapleys = sum_over_all_subsets / np.math.factorial(length_of_data)
    return  shapleys

In [11]:
def calculate_l_1_distance(vector_a, vector_b):
    a = np.array(vector_a)
    b = np.array(vector_b)
    return np.linalg.norm((a - b), ord=1)


In [64]:
def getOrderedValues(values, data_len):
    ordered_values = []
    for i in range(data_len):
        shapley_of_datum = [k[0][i] for k in values]
        ordered_values.append(shapley_of_datum)
    return ordered_values

In [65]:
def calculate_distance_of_shap_and_mc_to_ground_truth_shapleys(data):
    maximum_likelihood_calculator_d_1 = ExplainableMaximumLikelihoodCalculator(data,
                                                    ref_neanderthal_file_names=neanderthals_reference_file_names,
                                                    ref_sapien_file_names=sapiens_reference_file_names,
                                                    ref_denisovan_file_names=denisovan_reference_file_names,
                                                    path_to_substitution_matrix=path_to_frequencies_table,
                                                    number_of_jobs=-1)
    ground_truth_shapleys = []
    for i in range(len(data)):
        ground_truth_shapleys.append(calculate_shapley_values_for_datum(i,len(data),maximum_likelihood_calculator_d_1))
    shaps = maximum_likelihood_calculator_d_1.calculate_shapley_values()
    shaps_ordered = getOrderedValues(shaps, len(data))
    shapley_values_monte_carlo = maximum_likelihood_calculator_d_1.estimate_shapley_values(number_of_samples_per_read = 20)[1] # for the not scaled values
    mc_ordered = getOrderedValues(shapley_values_monte_carlo, len(data))
    sum_of_distances_shap = 0
    for i in range(len(data)):
        sum_of_distances_shap += calculate_l_1_distance(shaps_ordered[i], ground_truth_shapleys[i])
    sum_of_distances_mc = 0
    for i in range(len(data)):
        sum_of_distances_mc += calculate_l_1_distance(mc_ordered[i], ground_truth_shapleys[i])
    return (sum_of_distances_shap, sum_of_distances_mc)
    
    

In [66]:
sapiens = 2
neanderthals = 4
denisovans = 2
results = []
for i in range(3):
    dataset = generate_sample(sapiens, neanderthals, denisovans)
    distance = calculate_distance_of_shap_and_mc_to_ground_truth_shapleys(data=dataset)
    results.append(distance)
    print(distance)
    

Loading sequences and calculating alignments to all references, this might take a while. Number of reads:  8
start working on read number 0
1


  0%|          | 0/1 [00:00<?, ?it/s]

1
254
Start working on read number 0 in processId 1073
Start working on read number 1 in processId 1074
Start working on read number 2 in processId 1075
Start working on read number 3 in processId 1076Start working on read number 4 in processId 1077

Start working on read number 5 in processId 1078
Start working on read number 7 in processId 1080Start working on read number 6 in processId 1079

(0.27064285714285596, 1.1517380952380953)
Loading sequences and calculating alignments to all references, this might take a while. Number of reads:  8
start working on read number 0
1


  0%|          | 0/1 [00:00<?, ?it/s]

1
254
Start working on read number 0 in processId 1164
Start working on read number 1 in processId 1165
Start working on read number 2 in processId 1166
Start working on read number 3 in processId 1167Start working on read number 4 in processId 1168
Start working on read number 6 in processId 1170Start working on read number 7 in processId 1171Start working on read number 5 in processId 1169



(0.2597976190476204, 1.659428571428571)
Loading sequences and calculating alignments to all references, this might take a while. Number of reads:  8
start working on read number 0
1


  0%|          | 0/1 [00:00<?, ?it/s]

1
254
Start working on read number 0 in processId 1255
Start working on read number 1 in processId 1256
Start working on read number 2 in processId 1257
Start working on read number 3 in processId 1258
Start working on read number 5 in processId 1260Start working on read number 4 in processId 1259Start working on read number 6 in processId 1261


Start working on read number 7 in processId 1262
(0.23052380952380921, 1.6210119047619038)


In [67]:
results

[(0.27064285714285596, 1.1517380952380953),
 (0.2597976190476204, 1.659428571428571),
 (0.23052380952380921, 1.6210119047619038)]